In [3]:
import pandas as pd
import numpy as np
import sklearn
import sklearn.cluster
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import random
from ciu import determine_ciu
import six
import sys
import os
sys.modules['sklearn.externals.six'] = six
from skrules import SkopeRules
import openml
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


In [4]:
df_meta = pd.read_csv('metadata_merged.csv', index_col=0)
df_meta=df_meta.drop(columns= 'best_technique')

df_lime=pd.read_csv('records_lime.csv', index_col=0).drop(columns = 'lime_Dataset')
df_ciu=pd.read_csv('records_ciu.csv', index_col=0).drop(columns = 'ciu_Dataset')
df_anchor=pd.read_csv('records_anchor.csv', index_col=0).drop(columns = 'anchor_Dataset')

In [5]:
import pandas as pd

df_meta = pd.read_csv('metadata_merged.csv', index_col=0)
df_meta=df_meta.drop(columns= 'best_technique')

df_lime = pd.read_csv('records_lime.csv', index_col=0).drop(columns='lime_Dataset')
df_ciu = pd.read_csv('records_ciu.csv', index_col=0).drop(columns='ciu_Dataset')
df_anchor = pd.read_csv('records_anchor.csv', index_col=0).drop(columns='anchor_Dataset')

merged_df = df_lime.merge(df_ciu, on='folder').merge(df_anchor, on='folder')

criteria = ['Fidelity', 'Identity']
criteria_inverse = ['Separability', 'Speed']    

def assign_scores(row, crit, ascending=True):
    values = [row['lime_' + crit], row['ciu_' + crit], row['anchor_' + crit]]
    scores = pd.Series(values).rank(ascending=ascending, method='dense')
    return scores.values

for crit in criteria:
    scores = merged_df.apply(assign_scores, axis=1, args=(crit, False))
    merged_df[['lime_' + crit + '_score', 'ciu_' + crit + '_score', 'anchor_' + crit + '_score']] = pd.DataFrame(scores.tolist(), index=merged_df.index)

for crit in criteria_inverse:
    scores = merged_df.apply(assign_scores, axis=1, args=(crit, True))
    merged_df[['lime_' + crit + '_score', 'ciu_' + crit + '_score', 'anchor_' + crit + '_score']] = pd.DataFrame(scores.tolist(), index=merged_df.index)

score_columns = [col for col in merged_df.columns if 'score' in col]
merged_df['lime'] = merged_df[[col for col in score_columns if 'lime' in col]].sum(axis=1)
merged_df['ciu'] = merged_df[[col for col in score_columns if 'ciu' in col]].sum(axis=1)
merged_df['anchor'] = merged_df[[col for col in score_columns if 'anchor' in col]].sum(axis=1)

merged_df['best_dataset'] = merged_df[['lime', 'ciu', 'anchor']].idxmax(axis=1)

selected_datasets = merged_df[['folder', 'best_dataset']]
selected_datasets=selected_datasets.merge(df_meta, on='folder')
selected_datasets = selected_datasets.drop(columns='folder')


X = selected_datasets.drop(['best_dataset'], axis=1)
y = selected_datasets['best_dataset']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train, y_train)

y_pred = rf_classifier.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

         ciu       0.20      0.33      0.25         3
        lime       0.33      0.20      0.25         5

    accuracy                           0.25         8
   macro avg       0.27      0.27      0.25         8
weighted avg       0.28      0.25      0.25         8



In [6]:
def predict_best_dataset(input_data):

    input_df = pd.DataFrame([input_data])
    input_df = input_df[X.columns]

    prediction = rf_classifier.predict(input_df)

    return prediction[0]

In [16]:
input_data = df_meta.drop(columns='folder').iloc[30].to_dict()
input_data_formatted = {k: round(v, 2) for k, v in input_data.items()}
input_data_formatted


predicted_dataset = predict_best_dataset(input_data_formatted)
print("Predicted Best model for your cause, considering your dataset is", predicted_dataset.upper())

Predicted Best model for your cause, considering your dataset is ANCHOR


In [15]:
input_data = {'attr_to_inst': 0.74,
 'cat_to_num': 0.1,
 'freq_class.mean': 0.5,
 'freq_class.sd': 0.44,
 'inst_to_attr': 1,
 'max.mean': 0.595,
 'max.sd': 1.150,
 'min.mean': -0.8,
 'min.sd': 1.1,
 'nr_cor_attr':10.1,
 'nr_norm': 4.0,
 'sd.mean': 0.40,
 'sd.sd': 0.811}

input_data_formatted = {k: round(v, 2) for k, v in input_data.items()}


predicted_dataset = predict_best_dataset(input_data_formatted)
print("Predicted Best model for your cause, considering your dataset is", predicted_dataset.upper())

Predicted Best model for your cause, considering your dataset is LIME
